In [1]:
from baseline_cnn import *
from baseline_cnn import BasicCNN
import torch.nn as nn
import torch.optim as optim
import time
import pathlib
from evaluation import Evaluation
%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 32          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing


#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),transforms.ToTensor()])



# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = BasicCNN()
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = nn.MultiLabelSoftMarginLoss() #TODO - loss criteria are defined in the torch.nn package
criterion = nn.BCELoss()
#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters()) #TODO - optimizers are defined in the torch.optim package

CUDA is supported
Model on CUDA? True


In [2]:
print(model)

BasicCNN(
  (conv1): Conv2d(1, 12, kernel_size=(8, 8), stride=(1, 1))
  (conv1_normed): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 10, kernel_size=(8, 8), stride=(1, 1))
  (conv2_normed): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(10, 8, kernel_size=(6, 6), stride=(1, 1))
  (conv3_normed): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1928648, out_features=128, bias=True)
  (fc1_normed): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=14, bias=True)
)


In [3]:
# Set up folder for model saving
model_path = '{}/models/baseline/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
model_pathlib = pathlib.Path(model_path)
if not model_pathlib.exists():
    pathlib.Path(model_pathlib).mkdir(parents=True, exist_ok=True)

#save loss in files for futher usage.
loss_path = '{}/losses/baseline/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
loss_pathlib = pathlib.Path(loss_path)
if not loss_pathlib.exists():
    pathlib.Path(loss_pathlib).mkdir(parents=True, exist_ok=True)

In [4]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
loss_val_list = []
loss_val_min = float('inf')
N = 50
# Begin training procedure
for epoch in range(num_epochs):

    
    N_minibatch_loss = 0.0
    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
#         if minibatch_count == 100:
#             break
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print('training',minibatch_count,loss)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()
        
        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement validation
        if minibatch_count % N == 0:
            #switch to evaluate mode
            model.eval()
            with torch.no_grad():
                loss_val = 0
                for count_val, (images_val, labels_val) in enumerate(val_loader):
#                     if count_val ==10:
#                         break
                    images_val, labels_val = images_val.to(computing_device), labels_val.to(computing_device)
                    outputs_val = model(images_val)
                    loss_val += criterion(outputs_val, labels_val)
#                     print('val',count_val, (loss_val/(count_val+1)))
                loss_val /= count_val
                print('val',minibatch_count,loss_val)
                loss_val_list.append(loss_val.item())
                if loss_val < loss_val_min:
                    model_name = "epoch_{}-batch_{}-loss_{}-{}.pt".format(epoch, minibatch_count, loss_val, time.strftime("%Y%m%d-%H%M%S"))
                    torch.save(model.state_dict(), os.path.join(model_path, model_name))
                    loss_val_min = loss_val
                    
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss.item())
            N_minibatch_loss = 0.0
            
        prefix = 'baseline_loss_'
        with open(os.path.join(loss_path, prefix+"training.txt"), "w") as f:
            for s in total_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"training_ave.txt"), "w") as f:
            for s in avg_minibatch_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"val.txt"), "w") as f:
            for s in loss_val_list:
                f.write(str(s) +"\n")
            
    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


val 0 tensor(0.6937, device='cuda:0')
Epoch 1, average minibatch 0 loss: 0.014
val 50 tensor(0.2163, device='cuda:0')
Epoch 1, average minibatch 50 loss: 0.379
val 100 tensor(0.1869, device='cuda:0')
Epoch 1, average minibatch 100 loss: 0.205
val 150 tensor(0.1831, device='cuda:0')
Epoch 1, average minibatch 150 loss: 0.187
val 200 tensor(0.1824, device='cuda:0')
Epoch 1, average minibatch 200 loss: 0.180
val 250 tensor(0.1819, device='cuda:0')
Epoch 1, average minibatch 250 loss: 0.192
val 300 tensor(0.1815, device='cuda:0')
Epoch 1, average minibatch 300 loss: 0.184
val 350 tensor(0.1816, device='cuda:0')
Epoch 1, average minibatch 350 loss: 0.178
val 400 tensor(0.1810, device='cuda:0')
Epoch 1, average minibatch 400 loss: 0.189
val 450 tensor(0.1812, device='cuda:0')
Epoch 1, average minibatch 450 loss: 0.191
val 500 tensor(0.1800, device='cuda:0')
Epoch 1, average minibatch 500 loss: 0.186
val 550 tensor(0.1805, device='cuda:0')
Epoch 1, average minibatch 550 loss: 0.188


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()

## Inference

In [2]:
PATH = '/datasets/home/home-02/60/960/kshi/PA3/models/baseline/20190214-192159/epoch_0-batch_500-loss_0.1799965351819992-20190214-202710.pt'
model_test = BasicCNN()
model_test = model_test.to(computing_device)
model_test.load_state_dict(torch.load(PATH))

In [3]:
labels_all = []
predictions_all = []
for data in test_loader:
    images, labels = data
    
    images, labels = images.to(computing_device), labels.to(computing_device)
    labels_all.append(labels)
    output = model_test(images)
    predictions = output > 0.5
    predictions_all.append(predictions)

labels = torch.cat(labels_all,0)
predctions = torch.cat(predictions_all,0)

In [4]:
Evaluation(predctions.float(), labels)

TP [795.0, 180.0, 1328.0, 2734.0, 328.0, 275.0, 179.0, 367.0, 579.0, 357.0, 350.0, 70.0, 9.0, 8.0]
FP [6621.0, 6061.0, 6822.0, 11836.0, 6017.0, 4390.0, 12501.0, 5218.0, 11234.0, 15382.0, 15296.0, 7701.0, 351.0, 8065.0]
TN [11584.0, 13628.0, 10955.0, 4794.0, 13102.0, 14680.0, 7432.0, 14010.0, 8149.0, 4406.0, 4440.0, 12155.0, 19201.0, 12078.0]
FN [1181.0, 312.0, 1076.0, 817.0, 734.0, 836.0, 69.0, 586.0, 219.0, 36.0, 95.0, 255.0, 620.0, 30.0]


In [6]:
eval = Evaluation(predctions.float(), labels)
print(eval.accuracy())
print(eval.accuracy().mean())

TP [795.0, 180.0, 1328.0, 2734.0, 328.0, 275.0, 179.0, 367.0, 579.0, 357.0, 350.0, 70.0, 9.0, 8.0]
FP [6621.0, 6061.0, 6822.0, 11836.0, 6017.0, 4390.0, 12501.0, 5218.0, 11234.0, 15382.0, 15296.0, 7701.0, 351.0, 8065.0]
TN [11584.0, 13628.0, 10955.0, 4794.0, 13102.0, 14680.0, 7432.0, 14010.0, 8149.0, 4406.0, 4440.0, 12155.0, 19201.0, 12078.0]
FN [1181.0, 312.0, 1076.0, 817.0, 734.0, 836.0, 69.0, 586.0, 219.0, 36.0, 95.0, 255.0, 620.0, 30.0]
tensor([0.6134, 0.6842, 0.6086, 0.3730, 0.6655, 0.7410, 0.3771, 0.7124, 0.4325,
        0.2360, 0.2374, 0.6058, 0.9519, 0.5989], device='cuda:0',
       dtype=torch.float64)
tensor(0.5598, device='cuda:0', dtype=torch.float64)


In [7]:
eval.precision()

tensor([0.1072, 0.0288, 0.1629, 0.1876, 0.0517, 0.0589, 0.0141, 0.0657, 0.0490,
        0.0227, 0.0224, 0.0090, 0.0250, 0.0010], device='cuda:0',
       dtype=torch.float64)

In [8]:
eval.recall()

tensor([0.4023, 0.3659, 0.5524, 0.7699, 0.3089, 0.2475, 0.7218, 0.3851, 0.7256,
        0.9084, 0.7865, 0.2154, 0.0143, 0.2105], device='cuda:0',
       dtype=torch.float64)